##### Copyright 2024 Google LLC.

In [11]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Model tuning with Python

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs/model-tuning/python"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  </td>
    <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/model-tuning/python.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/model-tuning/python.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this notebook, you'll learn how to get started with the tuning service using the Python client library for the Gemini API. Here, you'll learn how to tune the text model behind the Gemini API's text generation service.

## Setup

### Install the client library

In [12]:
!pip install -q google-generativeai

### Import libraries

In [13]:
import google.generativeai as genai

In [14]:
from google.colab import userdata
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

You can check you existing tuned models with the `genai.list_tuned_model` method.

In [15]:
for i, m in zip(range(5), genai.list_tuned_models()):
  print(m.name)

tunedModels/generate-num-7238
tunedModels/generate-num-9560
tunedModels/resume-assessment-2895
tunedModels/resume-assessment-4337
tunedModels/resume-assessment-7379


## Create tuned model

To create a tuned model, you need to pass your dataset to the model in the `genai.create_tuned_model` method. You can do this be directly defining the input and output values in the call or importing from a file into a dataframe to pass to the method.

For this example, you will tune a model to generate the next number in the sequence. For example, if the input is `1`, the model should output `2`. If the input is `one hundred`, the output should be `one hundred one`.

In [16]:
base_model = [
    m for m in genai.list_models()
    if "createTunedModel" in m.supported_generation_methods and
    "flash" in m.name][0]
base_model

Model(name='models/gemini-1.5-flash-001-tuning',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash 001 Tuning',
      description=('Version of Gemini 1.5 Flash that supports tuning, our fast and versatile '
                   'multimodal model for scaling across diverse tasks, released in May of 2024.'),
      input_token_limit=16384,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens', 'createTunedModel'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)

In [17]:
# Synthesized training dataset for fine-tuning a candidate evaluation model.
# IMPORTANT: This data is AI-generated and requires human review and validation
# before use in actual fine-tuning.

training_dataset = [

  # Example 1: Junior Analyst vs. Junior Role (Good Match)
  [
    # text_input string
    """Job Description:
Role: Junior Data Analyst
Location: London, UK
Responsibilities: Assist senior analysts in collecting and cleaning data; Generate regular reports using predefined templates; Create basic dashboards in Excel and Power BI; Perform ad-hoc data extraction using SQL; Validate data accuracy under supervision.
Skills/Qualifications: Bachelor's degree in Statistics, Mathematics, Economics, or Computer Science; 0-1 years experience; Strong proficiency in Microsoft Excel (VLOOKUP, Pivot Tables); Basic understanding of SQL; Familiarity with Power BI or Tableau is a plus; Excellent attention to detail; Good communication skills.

--- Candidate Profile ---
[Candidate Name]
Junior Data Analyst | London, UK | [Phone Number] | [Email Address]

SUMMARY
Recent graduate with a BSc in Statistics seeking an entry-level Data Analyst position. Eager to apply analytical skills gained through coursework and projects. Proficient in Excel and SQL, with foundational knowledge of Power BI. Highly motivated, detail-oriented, and a quick learner.

EDUCATION
Bachelor of Science (BSc) - Statistics | September 2021 – June 2024
[University Name], London

PROJECTS
University Capstone Project - Sales Trend Analysis: Analyzed historical sales data for a mock retail company using Excel. Identified key trends using Pivot Tables and charts. Cleaned and validated data inputs. Presented findings to peers. (Excel, PowerPoint)
SQL for Data Science Course Project: Wrote SQL queries to extract and manipulate data from a sample database containing customer and order information. Practiced joins, aggregations, and subqueries. (SQL)

SKILLS
Data Analysis: Statistical Analysis (Coursework), Data Cleaning, Reporting
Programming/Databases: SQL (Basic), Excel (Advanced - VLOOKUP, Pivot Tables, Charts)
Visualization: Power BI (Basic - Coursework), Excel Charts
Other: Communication, Attention to Detail, Problem Solving (Coursework)

EXPERIENCE
(No professional experience)
""",
    # output string (Serialized JSON evaluation)
    """{
    "qualification_level": "Qualified",
    "interview_likelihood_score": "High (8/10)",
    "identified_mandatory_requirements": [
        "Bachelor's degree in relevant field (Stats, Math, Econ, CS)",
        "0-1 years experience",
        "Strong proficiency in Microsoft Excel",
        "Basic understanding of SQL",
        "Excellent attention to detail"
    ],
    "identified_preferred_qualifications": [
        "Familiarity with Power BI or Tableau"
    ],
    "justification_summary": {
        "mandatory_requirements_met": [
            "Possesses a relevant Bachelor's degree (BSc Statistics).",
            "Meets experience requirement (0 years).",
            "Strong Excel skills explicitly mentioned and demonstrated in project.",
            "Basic SQL understanding demonstrated via course project.",
            "Attention to detail listed as a skill."
        ],
        "mandatory_requirements_missing": [],
        "preferred_qualifications_present": [
            "Basic familiarity with Power BI mentioned from coursework."
        ],
        "key_strengths": [
            "Relevant degree in Statistics.",
            "Strong Excel skills confirmed by projects.",
            "Demonstrated initiative through SQL course project.",
            "Explicit mention of Power BI familiarity."
        ],
        "potential_gaps": [
            "Lacks professional work experience (expected for entry-level).",
            "Power BI/Tableau skills are basic."
        ]
    }
}"""
  ],

  # Example 2: Mid-Level Analyst vs. Marketing Analyst Role (Partial Match - Missing Marketing Tools)
  [
    # text_input string
    """Job Description:
Role: Marketing Data Analyst
Location: London, UK (Hybrid)
Responsibilities: Analyze marketing campaign performance across digital channels (PPC, SEO, Social Media, Email); Track website traffic and user behavior using Google Analytics; Develop and maintain marketing dashboards (Power BI); Segment customer data for targeted campaigns; Perform A/B testing analysis; Report on ROI and key marketing KPIs.
Skills/Qualifications: 3+ years experience in data analysis, preferably in a marketing context; Strong SQL skills; Proficiency in Google Analytics (GA4); Experience with marketing automation platforms (e.g., HubSpot, Marketo) is a plus; Advanced skills in Power BI or Tableau; Excellent analytical and communication skills.

--- Candidate Profile ---
[Candidate Name]
Data Analyst | London, UK | [Phone Number] | [Email Address]

SUMMARY
Data Analyst with 4 years of experience specializing in financial data analysis and reporting. Proven ability to translate complex data into actionable insights using SQL, Python, and Power BI. Seeking to leverage analytical skills in a new industry. Strong problem-solving and stakeholder communication abilities.

EXPERIENCE
Data Analyst | [Financial Services Company], London | July 2020 – Present
- Developed SQL queries to extract and aggregate data from large financial databases for regulatory reporting.
- Built interactive Power BI dashboards to monitor portfolio performance and risk metrics, used by senior management.
- Used Python (Pandas, NumPy) for data cleaning, transformation, and ad-hoc analysis.
- Collaborated with finance teams to understand reporting requirements.
Data Analyst | [Consulting Firm], London | June 2019 – July 2020
- Supported client projects involving data migration and validation using Excel and SQL.
- Created reports and presentations summarizing findings for clients.

EDUCATION
Master of Science (MSc) - Data Science | [University Name], UK | 2018 - 2019
Bachelor of Arts (BA) - Economics | [University Name], UK | 2015 - 2018

SKILLS
Data Analysis: Financial Analysis, Reporting, Data Validation, KPI Monitoring (Financial)
Programming/Databases: SQL (Advanced), Python (Pandas, NumPy), Excel (Advanced)
Visualization: Power BI (Advanced)
Other: Stakeholder Communication, Problem Solving
""",
    # output string (Serialized JSON evaluation)
    """{
    "qualification_level": "Potentially Qualified",
    "interview_likelihood_score": "Medium (5/10)",
    "identified_mandatory_requirements": [
        "3+ years experience in data analysis",
        "Strong SQL skills",
        "Proficiency in Google Analytics (GA4)",
        "Advanced skills in Power BI or Tableau"
    ],
    "identified_preferred_qualifications": [
        "Experience in a marketing context",
        "Experience with marketing automation platforms (e.g., HubSpot, Marketo)"
    ],
    "justification_summary": {
        "mandatory_requirements_met": [
            "Meets experience requirement (4 years).",
            "Strong/Advanced SQL skills demonstrated.",
            "Advanced skills in Power BI demonstrated."
        ],
        "mandatory_requirements_missing": [
            "Lacks explicitly stated experience with Google Analytics (GA4).",
            "Experience is primarily in finance, not marketing context.",
            "No mention of marketing automation platforms."
        ],
        "preferred_qualifications_present": [],
        "key_strengths": [
            "Strong SQL and Power BI expertise.",
            "Solid Python skills for data manipulation.",
            "Proven experience in reporting and dashboarding.",
            "Relevant Master's degree in Data Science."
        ],
        "potential_gaps": [
            "Significant gap in mandatory marketing-specific tools (Google Analytics).",
            "Lack of experience in analyzing marketing campaign data (PPC, SEO, Social).",
            "No experience with marketing automation platforms.",
            "Domain experience is in finance, not marketing."
        ]
    }
}"""
  ],

  # Example 3: Senior Analyst (Darren Gidado) vs. Lead Data Scientist Role (ML Focus - Good Match)
  [
    # text_input string
    """Job Description:
Role: Lead Data Scientist (Machine Learning)
Location: London, UK
Responsibilities: Lead the design, development, and deployment of machine learning models to solve business problems; Mentor junior data scientists; Research and implement cutting-edge ML algorithms and techniques; Collaborate with engineering teams to integrate models into production systems; Define project requirements and success metrics; Present complex findings to technical and non-technical stakeholders.
Skills/Qualifications: 5+ years experience in Data Science with a focus on Machine Learning; MSc or PhD in Computer Science, Statistics, or related quantitative field; Proven experience building and deploying ML models (e.g., classification, regression, clustering, NLP, recommendation systems); Strong programming skills in Python (including libraries like Scikit-learn, TensorFlow/PyTorch, Pandas, NumPy); Experience with cloud platforms (AWS, GCP, or Azure); Strong SQL skills; Excellent communication and leadership skills.

--- Candidate Profile ---
[Full text of Candidate CV (Darren Gidado) goes here...]
""",
    # output string (Serialized JSON evaluation)
    """{
    "qualification_level": "Highly Qualified",
    "interview_likelihood_score": "High (9/10)",
    "identified_mandatory_requirements": [
        "5+ years experience in Data Science/Analysis with ML focus",
        "Proven experience building and deploying ML models (classification, regression, NLP, recommendations mentioned)",
        "Strong programming skills in Python (Scikit-learn, Pandas, NumPy mentioned)",
        "Experience with cloud platforms (AWS, GCP, Azure mentioned)",
        "Strong SQL skills",
        "Excellent communication skills"
    ],
    "identified_preferred_qualifications": [
        "MSc or PhD in relevant field",
        "Experience mentoring junior team members",
        "Experience with TensorFlow/PyTorch",
        "Leadership skills"
    ],
    "justification_summary": {
        "mandatory_requirements_met": [
            "Meets 5+ years experience requirement in data roles with significant ML project work.",
            "Proven experience building ML models (Airbnb predictor, Disaster Response, IBM Recommendations, Starbucks Rewards) covering prediction, classification, NLP, recommendations.",
            "Strong Python skills demonstrated across CV (Pandas, NumPy, Scikit-learn, XGBoost, NLTK, Flask).",
            "Experience with AWS, GCP, Azure listed under skills.",
            "Strong SQL skills demonstrated through experience and certifications.",
            "Excellent communication implied via stakeholder management and client work."
        ],
        "mandatory_requirements_missing": [],
        "preferred_qualifications_present": [
            "Holds Data Scientist Nanodegree (relevant advanced training, though not MSc/PhD).",
            "Senior Data Analyst title implies seniority, potentially some informal leadership/mentoring.",
            "Experience aligns well with leadership needs (driving results, cross-functional work)."
        ],
        "key_strengths": [
            "Extensive hands-on experience building diverse ML models with quantifiable results/accuracy.",
            "Strong proficiency in Python and relevant ML libraries (Scikit-learn, NLP tools).",
            "Experience across multiple cloud platforms.",
            "Proven ability to deliver complex projects and work with stakeholders.",
            "Holds relevant Nanodegrees in Data Analysis and Data Science."
        ],
        "potential_gaps": [
            "Does not hold an MSc or PhD (Nanodegrees are valuable but may not meet requirement if strictly enforced).",
            "No explicit mention of TensorFlow/PyTorch.",
            "Explicit mentoring/team leadership experience is not detailed, though implied by senior title."
        ]
    }
}"""
  ],

  # Example 4: BI Developer vs. Data Analyst Role (Focus on Viz - Good Match)
  [
    # text_input string
    """Job Description:
Role: Data Analyst (Reporting & Visualization)
Location: Manchester, UK (Remote option)
Responsibilities: Design, develop, and maintain insightful dashboards and reports using Tableau and Power BI; Translate business requirements into technical specifications for reports; Optimize SQL queries for report performance; Ensure data accuracy and consistency in reports; Train end-users on dashboard usage.
Skills/Qualifications: 3+ years experience focused on BI development or data visualization; Expert-level skills in Tableau and/or Power BI; Strong SQL skills for data extraction and manipulation; Experience gathering requirements from business users; Excellent visual design and communication skills.

--- Candidate Profile ---
[Candidate Name]
BI Developer | Manchester, UK | [Phone Number] | [Email Address]

SUMMARY
Experienced BI Developer with 4 years specializing in creating interactive dashboards and reports using Tableau and Power BI. Passionate about data visualization and enabling data-driven decisions. Strong SQL skills and experience working directly with business stakeholders to define requirements.

EXPERIENCE
BI Developer | [Retail Company], Manchester | Jan 2021 – Present
- Developed and maintain a suite of Tableau dashboards monitoring sales, inventory, and customer behaviour, used across the company.
- Migrated legacy reports from Excel to Power BI, improving efficiency and interactivity.
- Wrote complex SQL queries to source data from various databases (SQL Server, Snowflake).
- Gathered reporting requirements from marketing, sales, and operations teams.
- Conducted training sessions for end-users on new dashboards.
Junior BI Analyst | [Tech Startup], Manchester | Dec 2019 – Jan 2021
- Assisted senior developers in building Tableau dashboards.
- Performed data validation and testing for reports.
- Handled ad-hoc data requests using SQL.

EDUCATION
Bachelor of Science (BSc) - Business Information Systems | [University Name], UK | 2016 - 2019

SKILLS
Visualization: Tableau (Expert), Power BI (Advanced)
Databases: SQL (Advanced - SQL Server, Snowflake)
Data Analysis: Reporting, Dashboarding, Requirements Gathering, Data Validation
Other: Communication, Training, Visual Design Principles
""",
    # output string (Serialized JSON evaluation)
    """{
    "qualification_level": "Highly Qualified",
    "interview_likelihood_score": "High (9/10)",
    "identified_mandatory_requirements": [
        "3+ years experience focused on BI development or data visualization",
        "Expert-level skills in Tableau and/or Power BI",
        "Strong SQL skills",
        "Experience gathering requirements from business users",
        "Excellent visual design and communication skills"
    ],
    "identified_preferred_qualifications": [
        "Experience training end-users"
    ],
    "justification_summary": {
        "mandatory_requirements_met": [
            "Meets experience requirement (4 years in BI roles).",
            "Expertise in Tableau and Power BI explicitly stated and demonstrated through extensive dashboard development.",
            "Strong/Advanced SQL skills shown via writing complex queries across different DBs.",
            "Direct experience gathering requirements from various business teams.",
            "Visual design principles listed as a skill, communication strong."
        ],
        "mandatory_requirements_missing": [],
        "preferred_qualifications_present": [
            "Explicitly mentions conducting training sessions for end-users."
        ],
        "key_strengths": [
            "Deep expertise in core visualization tools (Tableau, Power BI).",
            "Strong SQL skills across multiple database types.",
            "Proven experience translating business needs into BI solutions.",
            "Experience in the full report/dashboard lifecycle from requirements to training."
        ],
        "potential_gaps": [
            "Experience primarily focused on visualization/reporting, potentially less on broader data analysis if required beyond the JD scope."
        ]
    }
}"""
  ],

  # Example 5: Data Engineer vs. Data Analyst Role (Mismatch - Overqualified/Different Focus)
  [
    # text_input string
    """Job Description:
Role: Data Analyst
Location: London, UK
Responsibilities: Analyze sales and operational data to identify trends; Create reports and dashboards for management review (using Excel and Power BI); Support ad-hoc data requests from various departments; Clean and prepare data for analysis using SQL and potentially Python; Present findings clearly.
Skills/Qualifications: 2+ years experience as a Data Analyst; Proficiency in SQL and Excel; Experience with Power BI; Good communication skills; Bachelor's degree preferred.

--- Candidate Profile ---
[Candidate Name]
Senior Data Engineer | London, UK | [Phone Number] | [Email Address]

SUMMARY
Senior Data Engineer with 7 years of experience designing, building, and optimizing large-scale data pipelines and infrastructure on AWS. Expertise in Python, Spark, SQL, Airflow, and various database technologies (Redshift, PostgreSQL). Proven ability to handle terabytes of data and improve data processing efficiency. Seeking a challenging role focused on data infrastructure and architecture.

EXPERIENCE
Senior Data Engineer | [E-commerce Giant], London | 2019 – Present
- Designed and implemented scalable ETL/ELT pipelines using Python, Spark, and Airflow on AWS (S3, Redshift, EMR).
- Optimized data warehouse performance, reducing query times by 40%.
- Developed data quality frameworks and monitoring systems.
- Collaborated with data scientists and analysts to provide reliable data sources.
- Mentored junior engineers.
Data Engineer | [Media Company], London | 2017 – 2019
- Built data pipelines using Python and SQL.
- Managed PostgreSQL databases.
- Supported data warehousing initiatives.

EDUCATION
Master of Engineering (MEng) - Computer Engineering | [University Name], UK | 2012 - 2017

SKILLS
Data Engineering: ETL/ELT Pipelines, Data Modelling, Data Warehousing, Performance Optimization, Data Quality Frameworks
Programming/Databases: Python (Advanced - Spark, Airflow), SQL (Advanced), PostgreSQL, Redshift
Cloud: AWS (S3, Redshift, EMR, Glue)
Other: Mentoring, System Design
""",
    # output string (Serialized JSON evaluation)
    """{
    "qualification_level": "Not Qualified",
    "interview_likelihood_score": "Low (1/10)",
    "identified_mandatory_requirements": [
        "2+ years experience as a Data Analyst",
        "Proficiency in SQL and Excel",
        "Experience with Power BI",
        "Good communication skills"
    ],
    "identified_preferred_qualifications": [
        "Bachelor's degree"
    ],
    "justification_summary": {
        "mandatory_requirements_met": [
            "Exceeds experience requirement (7 years, though in Data Engineering).",
            "Advanced SQL proficiency implied.",
            "Holds a Master's degree (exceeds preferred Bachelor's).",
            "Communication skills likely present given senior role."
        ],
        "mandatory_requirements_missing": [
            "Lacks demonstrated experience specifically as a Data Analyst performing core analyst tasks (reporting, dashboarding for business users).",
            "No mention of Excel or Power BI proficiency; focus is on engineering tools (Python, Spark, Airflow, AWS).",
            "Role focus mismatch: Candidate is a Senior Data Engineer focused on infrastructure, pipelines, and optimization, while the role is a Data Analyst focused on reporting and business insights."
        ],
        "preferred_qualifications_present": [
             "Holds a Master's degree."
        ],
        "key_strengths": [
            "Deep technical expertise in data engineering tools and cloud platforms (Python, Spark, SQL, AWS).",
            "Extensive experience building and optimizing data infrastructure.",
            "Proven ability to handle large datasets.",
            "Mentoring experience."
        ],
        "potential_gaps": [
            "Complete mismatch in role focus and primary responsibilities.",
            "Lack of demonstrated skills in core analyst tools required by the JD (Excel, Power BI).",
            "Candidate appears significantly overqualified in engineering terms and likely seeking a different type of role (as stated in summary)."
        ]
    }
}"""
  ],

  # Example 6: Analyst with R vs. Python Role (Tool Mismatch)
  [
    # text_input string
    """Job Description:
Role: Data Analyst (Python Focus)
Location: London, UK
Responsibilities: Utilize Python (Pandas, NumPy, Scikit-learn) for complex data analysis, modeling, and automation; Develop and maintain analytical scripts; Build visualizations using Python libraries (Matplotlib, Seaborn, Plotly); Extract data using SQL; Collaborate with product teams to provide insights.
Skills/Qualifications: 3+ years experience in data analysis; Strong Python programming skills for data analysis (Pandas, NumPy required); Experience with ML libraries (Scikit-learn) is a strong plus; Proficiency in SQL; Degree in quantitative field.

--- Candidate Profile ---
[Candidate Name]
Data Analyst | London, UK | [Phone Number] | [Email Address]

SUMMARY
Data Analyst with 4 years experience in the healthcare sector, specializing in statistical analysis and reporting using R and SQL. Proven ability to derive insights from clinical trial data. Seeking a role applying analytical skills to new challenges. Adept at communicating findings to researchers and clinicians.

EXPERIENCE
Data Analyst | [Pharmaceutical Company], London | 2020 – Present
- Performed statistical analysis on clinical trial data using R.
- Developed R scripts for data cleaning, manipulation, and visualization (ggplot2).
- Created reports summarizing trial outcomes for regulatory submissions.
- Extracted data from clinical databases using SQL.
- Collaborated with study managers and statisticians.
Junior Analyst | [Healthcare Provider], London | 2019 – 2020
- Assisted with data collection and entry.
- Generated basic reports using Excel and R.

EDUCATION
Master of Science (MSc) - Biostatistics | [University Name], UK | 2017 - 2019
Bachelor of Science (BSc) - Mathematics | [University Name], UK | 2014 - 2017

SKILLS
Data Analysis: Statistical Analysis (Clinical Trials), Reporting
Programming/Databases: R (Advanced - ggplot2), SQL (Proficient), Excel (Intermediate)
Visualization: R (ggplot2)
Domain: Healthcare, Clinical Trials
Other: Communication
""",
    # output string (Serialized JSON evaluation)
    """{
    "qualification_level": "Not Qualified",
    "interview_likelihood_score": "Low (2/10)",
    "identified_mandatory_requirements": [
        "3+ years experience in data analysis",
        "Strong Python programming skills for data analysis (Pandas, NumPy required)",
        "Proficiency in SQL",
        "Degree in quantitative field"
    ],
    "identified_preferred_qualifications": [
        "Experience with ML libraries (Scikit-learn)"
    ],
    "justification_summary": {
        "mandatory_requirements_met": [
            "Meets experience requirement (4 years).",
            "Proficient in SQL.",
            "Holds relevant degrees (MSc Biostatistics, BSc Mathematics)."
        ],
        "mandatory_requirements_missing": [
            "Critically lacks the required strong Python programming skills (Pandas, NumPy). Candidate's expertise is explicitly in R.",
            "No demonstrated experience with Python visualization libraries (Matplotlib, Seaborn, Plotly)."
        ],
        "preferred_qualifications_present": [
             "While not Python's Scikit-learn, the candidate has strong statistical analysis background which is related to ML."
        ],
        "key_strengths": [
            "Strong statistical analysis skills, particularly in R.",
            "Experience in the healthcare/clinical trial domain.",
            "Proficient in SQL.",
            "Relevant quantitative degrees."
        ],
        "potential_gaps": [
            "Primary programming language (R) does not match the core requirement (Python).",
            "Lack of experience with the specific Python libraries required (Pandas, NumPy, visualization tools).",
            "No demonstrated ML experience using Python tools."
        ]
    }
}"""
  ],

  # Example 7: General Analyst vs. Financial Analyst Role (Domain Mismatch)
  [
      # text_input string
      """Job Description:
Role: Financial Data Analyst
Location: London, UK
Responsibilities: Analyze financial statements, investment performance, and market trends; Develop financial models for forecasting and valuation; Create reports and presentations for senior management and investors; Monitor financial KPIs and identify variances; Ensure data integrity in financial systems; Utilize SQL and advanced Excel for analysis.
Skills/Qualifications: 3+ years experience in financial analysis or a related role; Strong understanding of accounting principles and financial statements; Advanced financial modeling skills in Excel; Proficiency in SQL; Experience with financial data sources (e.g., Bloomberg, Refinitiv) is a plus; Bachelor's degree in Finance, Accounting, or Economics.

--- Candidate Profile ---
[Candidate Name]
Data Analyst | London, UK | [Phone Number] | [Email Address]

SUMMARY
Data Analyst with 4 years of experience analyzing operational and customer data for a large retail company. Expertise in SQL, Excel, and Tableau for reporting and identifying efficiency improvements. Seeking a challenging analytical role.

EXPERIENCE
Data Analyst | [Retail Company], London | 2020 – Present
- Analyzed sales data to identify regional trends and optimize stock levels using SQL and Excel.
- Developed Tableau dashboards to track customer purchasing behavior and loyalty program effectiveness.
- Identified £1M+ in potential cost savings through analysis of supply chain data.
- Prepared weekly operational performance reports for management.
- Collaborated with marketing team on campaign analysis (using internal data).
Junior Analyst | [Logistics Firm], London | 2019 – 2020
- Supported reporting tasks using Excel.
- Performed basic data cleaning and validation.

EDUCATION
Bachelor of Science (BSc) - Business Management | [University Name], UK | 2016 - 2019

SKILLS
Data Analysis: Operational Analysis, Customer Analysis, Reporting, Trend Analysis
Programming/Databases: SQL (Proficient), Excel (Advanced)
Visualization: Tableau (Advanced)
Other: Communication, Problem Solving
""",
      # output string (Serialized JSON evaluation)
      """{
    "qualification_level": "Not Qualified",
    "interview_likelihood_score": "Low (3/10)",
    "identified_mandatory_requirements": [
        "3+ years experience in financial analysis or related role",
        "Strong understanding of accounting principles and financial statements",
        "Advanced financial modeling skills in Excel",
        "Proficiency in SQL",
        "Bachelor's degree in Finance, Accounting, or Economics"
    ],
    "identified_preferred_qualifications": [
        "Experience with financial data sources (e.g., Bloomberg, Refinitiv)"
    ],
    "justification_summary": {
        "mandatory_requirements_met": [
            "Meets 3+ years experience in data analysis (though not financial).",
            "Proficient in SQL.",
            "Advanced Excel skills demonstrated (though not specifically financial modeling)."
        ],
        "mandatory_requirements_missing": [
            "Lacks demonstrated experience specifically in *financial* analysis.",
            "No evidence of understanding accounting principles or financial statements.",
            "No evidence of advanced *financial* modeling skills in Excel.",
            "Degree is in Business Management, not Finance/Accounting/Economics.",
            "No experience with financial data sources."
        ],
        "preferred_qualifications_present": [],
        "key_strengths": [
            "Strong analytical skills demonstrated in operational/customer context.",
            "Proficient in SQL and advanced Excel.",
            "Advanced Tableau skills for visualization.",
            "Proven ability to identify cost savings and analyze trends."
        ],
        "potential_gaps": [
            "Major gap in required financial domain knowledge (accounting, financial statements, financial modeling).",
            "Educational background does not match specific requirements.",
            "Lack of experience with typical financial data sources.",
            "Experience is operational/customer-focused, not finance-focused."
        ]
    }
}"""
  ],

  # Example 8: Analyst with Cloud vs. Role Requiring Cloud (Good Match)
  [
      # text_input string
      """Job Description:
Role: Cloud Data Analyst
Location: London, UK (Hybrid)
Responsibilities: Analyze large datasets stored in cloud environments (AWS S3, Redshift/Snowflake); Use SQL and Python for data extraction, transformation, and analysis within the cloud; Build dashboards using cloud-compatible tools (e.g., QuickSight, Power BI); Monitor data pipelines and data quality in the cloud; Collaborate with data engineers and business stakeholders.
Skills/Qualifications: 3+ years experience in data analysis; Strong SQL proficiency; Experience querying data in cloud data warehouses (Redshift, BigQuery, or Snowflake); Experience using Python for data analysis (Pandas); Familiarity with AWS, GCP, or Azure data services; Experience with cloud BI tools.

--- Candidate Profile ---
[Candidate Name]
Data Analyst | London, UK | [Phone Number] | [Email Address]

SUMMARY
Data Analyst with 4 years experience leveraging cloud technologies (AWS) for data analysis and reporting. Proficient in SQL, Python, Redshift, and Power BI. Skilled in handling large datasets and delivering insights to support business strategy.

EXPERIENCE
Data Analyst | [Tech Company - AWS Partner], London | 2020 – Present
- Analyzed user engagement data stored in AWS S3 and Redshift using SQL and Python (Pandas).
- Developed ETL scripts in Python to process data for analysis environment.
- Built Power BI dashboards connected to Redshift to visualize key product metrics.
- Monitored data quality and pipeline performance using AWS CloudWatch.
- Worked with product managers to define analysis requirements.
Junior Data Analyst | [SaaS Company], London | 2019 – 2020
- Extracted data from PostgreSQL using SQL.
- Created reports in Excel.
- Assisted with data cleaning tasks.

EDUCATION
Bachelor of Science (BSc) - Computer Science | [University Name], UK | 2016 - 2019

SKILLS
Data Analysis: Cloud Data Analysis, Reporting, Dashboarding, Data Quality Monitoring
Programming/Databases: SQL (Advanced), Python (Pandas), Redshift, PostgreSQL
Cloud: AWS (S3, Redshift, CloudWatch)
Visualization: Power BI (Advanced)
Other: Communication, Collaboration
""",
      # output string (Serialized JSON evaluation)
      """{
    "qualification_level": "Highly Qualified",
    "interview_likelihood_score": "High (9/10)",
    "identified_mandatory_requirements": [
        "3+ years experience in data analysis",
        "Strong SQL proficiency",
        "Experience querying data in cloud data warehouses (Redshift, BigQuery, or Snowflake)",
        "Experience using Python for data analysis (Pandas)",
        "Familiarity with AWS, GCP, or Azure data services",
        "Experience with cloud BI tools"
    ],
    "identified_preferred_qualifications": [],
    "justification_summary": {
        "mandatory_requirements_met": [
            "Meets 3+ years experience requirement (4 years).",
            "Strong/Advanced SQL proficiency demonstrated.",
            "Direct experience querying data in Redshift (cloud data warehouse).",
            "Experience using Python (Pandas) for analysis on cloud data.",
            "Strong familiarity with AWS data services (S3, Redshift, CloudWatch).",
            "Experience with Power BI connected to cloud data source (Redshift)."
        ],
        "mandatory_requirements_missing": [],
        "preferred_qualifications_present": [],
        "key_strengths": [
            "Direct hands-on experience analyzing data within AWS cloud environment.",
            "Proficiency in key tools required: SQL, Python (Pandas), Redshift, Power BI.",
            "Experience includes ETL and data quality monitoring in the cloud.",
            "Relevant Computer Science degree."
        ],
        "potential_gaps": [
            "Experience primarily focused on AWS; less explicit mention of GCP/Azure if those are critical.",
            "No mention of BigQuery or Snowflake specifically, but Redshift experience is highly relevant."
        ]
    }
}"""
  ],

  # Example 9: Senior Analyst vs. Senior Role (Good Match - Meets All)
  [
      # text_input string
      """Job Description:
Role: Senior Data Analyst
Location: London, UK
Responsibilities: Lead complex analytical projects from requirements gathering to presentation; Develop predictive models and perform advanced statistical analysis; Mentor junior analysts; Design and implement A/B tests; Own reporting for a key business area using SQL, Python, and Power BI; Communicate insights and recommendations to senior leadership.
Skills/Qualifications: 5+ years experience in data analysis; Expert SQL skills; Advanced Python skills for analysis and modeling (Pandas, NumPy, Scikit-learn); Proven experience with statistical analysis and predictive modeling; Expertise in Power BI or similar tool; Experience designing and analyzing A/B tests; Strong presentation and stakeholder management skills.

--- Candidate Profile ---
[Candidate Name]
Senior Data Analyst | London, UK | [Phone Number] | [Email Address]

SUMMARY
Senior Data Analyst with 6 years of experience driving business decisions through advanced analytics, predictive modeling, and A/B testing. Expertise in SQL, Python (inc. Scikit-learn), and Power BI. Proven ability to lead projects, mentor analysts, and present findings to executive leadership. Seeking a challenging senior role.

EXPERIENCE
Senior Data Analyst | [E-commerce Company], London | 2020 – Present
- Led development of customer churn prediction model using Python (Scikit-learn), reducing churn by 8%.
- Designed and analyzed A/B tests for website optimizations, leading to a 15% uplift in conversion rate.
- Owned all marketing analytics reporting, developing complex SQL queries and automated Power BI dashboards.
- Mentored two junior data analysts, providing guidance on technical skills and project work.
- Presented key findings and strategic recommendations to C-level executives monthly.
Data Analyst | [Same E-commerce Company], London | 2018 – 2020
- Conducted deep-dive analyses into customer behavior using SQL and Python.
- Built initial versions of marketing performance dashboards in Power BI.
- Supported A/B test setup and data collection.

EDUCATION
Master of Science (MSc) - Statistics | [University Name], UK | 2016 - 2018
Bachelor of Science (BSc) - Mathematics | [University Name], UK | 2013 - 2016

SKILLS
Data Analysis: Advanced Statistical Analysis, Predictive Modeling (Churn), A/B Testing, Reporting, Dashboarding
Programming/Databases: SQL (Expert), Python (Advanced - Pandas, NumPy, Scikit-learn)
Visualization: Power BI (Expert)
Other: Mentoring, Stakeholder Management (inc. C-level), Project Leadership, Communication, Presentation
""",
      # output string (Serialized JSON evaluation)
      """{
    "qualification_level": "Highly Qualified",
    "interview_likelihood_score": "Very High (10/10)",
    "identified_mandatory_requirements": [
        "5+ years experience in data analysis",
        "Expert SQL skills",
        "Advanced Python skills for analysis and modeling (Pandas, NumPy, Scikit-learn)",
        "Proven experience with statistical analysis and predictive modeling",
        "Expertise in Power BI or similar tool",
        "Experience designing and analyzing A/B tests",
        "Strong presentation and stakeholder management skills"
    ],
    "identified_preferred_qualifications": [
        "Experience mentoring junior analysts",
        "Experience presenting to senior leadership"
    ],
    "justification_summary": {
        "mandatory_requirements_met": [
            "Meets experience requirement (6 years).",
            "Expert SQL skills implied by owning reporting and developing complex queries.",
            "Advanced Python skills including Scikit-learn demonstrated via predictive modeling.",
            "Proven experience with statistical analysis (degree) and predictive modeling (churn model).",
            "Expertise in Power BI demonstrated via owning reporting and automated dashboards.",
            "Direct experience designing and analyzing A/B tests with positive results.",
            "Strong presentation/stakeholder skills shown via presenting to C-level executives."
        ],
        "mandatory_requirements_missing": [],
        "preferred_qualifications_present": [
            "Explicitly mentored two junior analysts.",
            "Presented findings directly to C-level executives."
        ],
        "key_strengths": [
            "Direct, proven experience in all mandatory technical areas (SQL, Python/Scikit-learn, Power BI, A/B testing).",
            "Strong track record of delivering impactful results (churn reduction, conversion uplift).",
            "Demonstrated leadership and mentoring capabilities.",
            "Experience communicating at the highest organizational levels.",
            "Relevant advanced degree in Statistics."
        ],
        "potential_gaps": [
            "None obvious based on the provided job description."
        ]
    }
}"""
  ],

  # Example 10: Analyst Lacking Specific Tool (e.g., Power BI) vs. Role Requiring It
  [
      # text_input string
      """Job Description:
Role: Data Analyst
Location: London, UK
Responsibilities: Analyze operational data to identify inefficiencies; Create and maintain performance dashboards exclusively in Power BI; Extract and clean data using SQL; Support business users with ad-hoc reporting requests; Document data definitions and report logic.
Skills/Qualifications: 2+ years experience in data analysis; Strong SQL skills; **Mandatory: Advanced Power BI skills (including DAX)**; Experience with data cleaning and validation; Good communication skills.

--- Candidate Profile ---
[Candidate Name]
Data Analyst | London, UK | [Phone Number] | [Email Address]

SUMMARY
Data Analyst with 3 years experience focused on data extraction, cleaning, and analysis using SQL and Python. Skilled in identifying data quality issues and providing accurate datasets for reporting teams. Experience primarily with Tableau for visualization. Seeking a role involving data quality and analysis.

EXPERIENCE
Data Analyst | [Data Quality Consultancy], London | 2021 – Present
- Developed SQL scripts to profile client data and identify quality issues (incompleteness, inconsistency).
- Used Python (Pandas) for complex data cleaning and transformation tasks.
- Created data quality reports and dashboards for clients using Tableau.
- Documented data lineage and quality rules.
- Liaised with client technical teams to understand data sources.
Junior Analyst | [Insurance Company], London | 2020 – 2021
- Supported data migration project, performing data validation using SQL and Excel.
- Handled basic data extraction requests.

EDUCATION
Bachelor of Science (BSc) - Information Management | [University Name], UK | 2017 - 2020

SKILLS
Data Analysis: Data Quality Analysis, Data Cleaning, Data Validation, Reporting (via Tableau)
Programming/Databases: SQL (Advanced), Python (Pandas), Excel (Intermediate)
Visualization: Tableau (Advanced)
Other: Documentation, Communication
""",
      # output string (Serialized JSON evaluation)
      """{
    "qualification_level": "Not Qualified",
    "interview_likelihood_score": "Low (2/10)",
    "identified_mandatory_requirements": [
        "2+ years experience in data analysis",
        "Strong SQL skills",
        "Mandatory: Advanced Power BI skills (including DAX)"
    ],
    "identified_preferred_qualifications": [
        "Experience with data cleaning and validation",
        "Good communication skills"
    ],
    "justification_summary": {
        "mandatory_requirements_met": [
            "Meets experience requirement (3 years).",
            "Strong/Advanced SQL skills demonstrated.",
            "Experience with data cleaning and validation.",
            "Good communication skills implied."
        ],
        "mandatory_requirements_missing": [
            "Critically lacks the mandatory Advanced Power BI skills (including DAX). Candidate's visualization experience is explicitly with Tableau."
        ],
        "preferred_qualifications_present": [
             "Experience with data cleaning and validation.",
             "Good communication skills implied."
        ],
        "key_strengths": [
            "Strong SQL skills.",
            "Experience in data quality analysis and data cleaning.",
            "Proficiency in Python (Pandas).",
            "Advanced skills in Tableau (alternative visualization tool)."
        ],
        "potential_gaps": [
            "Complete lack of the mandatory tool (Power BI / DAX). Experience is with a different tool (Tableau)."
        ]
    }
}"""
  ]

]

# You would then use this 'training_dataset' list in your fine-tuning script.
# Remember to replace placeholder text like '[Full text of...]' with actual data.

In [18]:
import random

name = f'generate-num-{random.randint(0,10000)}'
operation = genai.create_tuned_model(
    # You can use a tuned model here too. Set `source_model="tunedModels/..."`
    source_model=base_model.name,
    training_data=training_dataset,
    id=name,
    epoch_count=100,
    batch_size=4,
    learning_rate=0.001,
)

Your tuned model is immediately added to the list of tuned models, but its status is set to "creating" while the model is tuned.

In [19]:
model = genai.get_tuned_model(f'tunedModels/{name}')

model

TunedModel(name='tunedModels/generate-num-3057',
           source_model='models/gemini-1.5-flash-001-tuning',
           base_model='models/gemini-1.5-flash-001-tuning',
           display_name='',
           description='',
           temperature=1.0,
           top_p=0.95,
           top_k=64,
           state=<State.CREATING: 1>,
           create_time=datetime.datetime(2025, 4, 18, 23, 50, 9, 493470, tzinfo=datetime.timezone.utc),
           update_time=datetime.datetime(2025, 4, 18, 23, 50, 9, 493470, tzinfo=datetime.timezone.utc),
           tuning_task=TuningTask(start_time=None,
                                  complete_time=None,
                                  snapshots=[],
                                  hyperparameters=Hyperparameters(epoch_count=100,
                                                                  batch_size=4,
                                                                  learning_rate=0.001)),
           reader_project_numbers=None)

In [20]:
model.state

<State.CREATING: 1>

### Check tuning progress

Use `metadata` to check the state:

In [21]:
operation.metadata

total_steps: 250
tuned_model: "tunedModels/generate-num-3057"

Wait for the training to finish using `operation.result()`, or `operation.wait_bar()`

In [ ]:
import time

for status in operation.wait_bar():
  time.sleep(30)

  0%|          | 0/250 [00:00<?, ?it/s]

You can cancel your tuning job any time using the `cancel()` method. Uncomment the line below and run the code cell to cancel your job before it finishes.

In [ ]:
# operation.cancel()

Once the tuning is complete, you can view the loss curve from the tuning results. The [loss curve](https://ai.google.dev/gemini-api/docs/model-tuning#recommended_configurations) shows how much the model's predictions deviate from the ideal outputs.

In [ ]:
import pandas as pd
import seaborn as sns

model = operation.result()

snapshots = pd.DataFrame(model.tuning_task.snapshots)

sns.lineplot(data=snapshots, x = 'epoch', y='mean_loss')


## Evaluate your model

You can use the `genai.generate_content` method and specify the name of your model to test your model performance.

In [ ]:
model = genai.GenerativeModel(model_name=f'tunedModels/{name}')

In [ ]:
resume = """
Resume: ACCOUNT SERVICE REPRESENTATIVE II/TECH Summary Professional Account Manager trained in building strategic network ties and increasing salesrevenues. Cultivates and maintains strong customer relationships. Accomplishments Prospecting and cold callingExcellent communicatorStrong interpersonal skillsCustomer service-focusedTeam building expertiseTech-savvySkilled in MS OfficeProject managementProven sales track recordPromotional campaignsPowerPoint expertLeadership/communication skillsSkilled multi-taskerStrong organizational skillsStrong communicatorActive listening skillsExceptional customer service skillsSelf-motivated. Accomplishments. Named "Employee of the Month" in 02/2014 at C Spire.Winner of the "C Spire Employee of the Quarter" Award, 1st Quarter 2014.Maintained 100% Survey Rating entire tenure at Foshee Residential.Top 5 in New Accounts at Foshee Residential (2012)1st Place Southeast Regionals American Kickboxing Academy Tournament (3x) for ShogunMMA. Experience Account Service Representative II/Tech January 2012 to May 2014 Company Name － State Project Design & Development LeadMay 2014. Henderson-Thomas * Birmingham, Alabama. Discover, design and build new user experiences to drive the next generation of portals,applications and Web properties, including mobile. Design visually stunning, cutting-edgegraphics, animations and interactions for Web sites, portals and applications. Produce visualassets needed by development team. Contribute to building the Company brand on portals,applications, mobile apps and Web sites by executing user interface design consistently on alldeliverables. Refine low to high-fidelity wireframes into designs for the informationarchitecture, navigational models, process flows, page layout, information displays andinteractional components based on user research and in partnership with user experienceinteraction designers. Research and communicate trends and best-practices related to visualdesign and branding Every team member of the C Spire Crew is charged with bringing a strategic vision ofpersonalized wireless to life. Responsible for achieving sales goals and customer satisfactionstandards at designated retail locations. ASRs must demonstrate a strong aptitude andknowledge of all things technology personalized to each customer\'s needs, and project anenergetic and upbeat professional image. This position is also responsible for integratingtechnology into the consumer\'s life with amazing ease and effectiveness. ASR II is alsocharged with extra supervisory duties such as key holding and employee management andinventory auditing. Developing and maintaining in-store promotions, and coordinating withappropriate personnel Implementing and managing wireless sales events Provide positive. reinforcement and adult learning techniques to promote learning and skill improvement.Participated in "Fiber to The Home" installation\'s receiving basic knowledge of Telephone,Cable and Internet home installation. Successfully increased positivity and smiling in theworkplace. Promoted group activities and continuously met my quota and passed customerexperience surveys and mystery shops. Gained promotion to level II (key holder/assistantmanager) position based on experience and performance. January 2010 to December 2012 Company Name － City , State Responsibilities Supervise and assists staff to ensure they are well informed on their positionresponsibilities and are performing at or above expectations. Managers with leasing staff areresponsible for agents overall performance including leasing numbers and entering trafficTrain new staff as well as provide on-going training and feedback on performance Ensuresthat building and office are properly staffed at all times Approves time off request for staffEnsures staff provides necessary customer relations for superior customer satisfactionRoutinely inspect properties to ensure they are safe and to determine if repairs ormaintenance are needed Monitors property vendors to ensure quality service Process workorders and monitors direction of property maintenance operations to ensure compliance withcompany standards Respond to tenant requests and issues in a timely manner Maintaincustomer service standards and resident relations Complete leasing/administrativepaperwork Accountable for market surveys Keeps lease occupancies and rents at highestdesired level Review delinquencies and assist in collections May oversee the preparation ofbudgets, financial statements and periodic reports on the status of various properties, leaseexpirations and related matters Ensure marketready condition of vacant units and managesturn-over Coordinate lease terminations, preliminary inspections, and apartment turn-overProvides tours of apartments and community to potential residents for purpose of leasingapartments Coordinate outreach marketing and advertising Conduct the leasing process withprospects in absence of leasing consultant Completes all other projects or requests as directedby Supervison. Martial Arts Instructor/CLient January 2008 to May 2012 Company Name － State Instruct students in varied martial arts techniques Prepare and distribute detailed monthly,quarterly and yearly reports to clients. Provide required information and documents toclients. Prepare reports of client meetings to record the details for future reference. Submitclient meeting reports to manager or supervisor. Provide support and assistance tomanagement teams. Audit all client liaison active accounts. Revise submission and accountsclosure applications. Develop liaison with client service and collection department to providerequired information and acquire necessary documents. Revise and distribute contractualreferrals. Provide feedback to management on areas of improvements. Recommendimplementation of enhanced processes. Plan and schedule monthly meetings with clients.Monitor client satisfaction through follow up on solving client issues and problems. Initiatemeasures to research and resolve client raised issues. Collaborate with clients to identify andimplement value added services.Grew customer base from 13 accounts to 34 accounts in 2years. Education Bachelor of Science : Business (Entrepreneurship) , 2014 Auburn University － City , State , United States GPA: GPA: 3.87 Business (Entrepreneurship) Coursework in Business Administration and ManagementCoursework in Marketing and CommunicationsCoursework in Public Relations GPA: 3.87 High School Diploma : Math & Science , 2009 Alabama School of Math & Science * Auburn － State , United States GPA: Student government representativeHiQ MemberHonor Student2009 Dean\'s List Math & Science Student government representativeHiQ MemberHonor Student2009 Dean\'s List Skills advertising, arts, auditing, basic, branding, Business Administration, C, Cable, consultant, client, clients, customer relations, client liaison, direction, financial statements, image, employee management, managing, marketing, market, meetings, office, Monitors, next, page layout, personnel, processes, Project Design & Development, promotion, Public Relations, quality, receiving, repairs, Research, retail, sales, strategic, supervisor, supervisory, Telephone, user interface design, vision, Web sites\n\n

Job Description: **Important to note that this role is a Hybrid in-house role - NO remote opportunities available**Dear Fearless AF Marketing wizard,If you’ve been searching through countless lifeless job listings on Seek…Almost slipping into a deep coma from the “We’re looking for a highly experienced Marketing Strategist... blah blah blah” job adverts…And you’re looking for an exciting opportunity to join the fastest growing digital agency in the country…Then this will be the most important job listing you ever read.Here’s why…Firstly... stop what you\'re doing and watch thisOkay lets continue... We are a full-service digital marketing agency that is growing FAST! For the last three years, we’ve been ranked as the fastest-growing digital agency…And the 17th fastest growing company in the country by The Australian Financial Review…We\'re a world-class herd of digital strategists, direct response marketers, designers, growth hackers and developers, banded together to help you reach your growth goals.And this is just one of the reasons we’ve been named as one of the BEST PLACES TO WORK in Australia &amp; New Zealand by AFR and one of the Top 100 Coolest Companies in Australia by Anthill Magazine.Look, the list goes on and on…because we’ve seriously won more awards than you can shake a stick at…The Opportunity: You will be working directly with a portfolio of clients managing the communication and identifying improvements of their SEO and pay-per-click campaigns (Google AdWords, Facebook Ads &amp; YouTube Ads).Our team of specialists will build and manage the clients campaigns, allowing you to focus on diving deeper into each account to strategise how we can achieve even more ROI for the client. With our team of specialists at your disposal, we\'re looking for an analytical mind that is results driven and dedicated to seeing their clients dominate their market.What you\'ll do: Act as the main point of contact for the portfolio of online marketing clients, building a positive relationship as the representative for the company. Conduct strategy audits to identify further opportunities for your clients to scale their brand. Prepare monthly reports (with the super cool software we use) and make recommendations for ongoing improvements with the rest of the team. Analyse analytics to identify areas of opportunity and utilise conversion rate optimisation tactics to squeeze as much ROI as humanly possible out of each dollar spent. Educate clients on our SEO and PPC (Google AdWords, Facebook Ads &amp; YouTube Ads) strategy and tactics. Providing an exceptional customer experience with the ability to turn a negative into a positive. RequirementsOur dream candidate: Must have 1-2 years of agency experience in a similar role Intermediate knowledge and passion of digital marketing, including SEO, PPC (Google Ads, Facebook), Conversion Rate Optimisation, Sales Funnels and Growth Hacking. Intermediate level experience analysing data in Google Analytics, Shopify, Facebook and Google Ads platforms. A hunger for learning and enhancing your existing skills. Confident in up-selling and retaining clients. Highly organised and process-driven. Exceptional customer service is KEY! Google Analytics and Google Ads certified is a PLUS BenefitsThe juicy part: Salary plus generous uncapped commission structure - we want America\'s best talent and reward as such. Extensive training and full support - we take training very seriously and will provide you with the industry\'s very best training, giving you the knowledge and confidence needed for greatness. Amazing growth prospects and upward mobility in our fast-paced agency. INSANE events - You name it and we\'ve probably done it. Company trips away - such as glamping, Bali &amp; Thailand. Hybrid Working - because sometimes you just want to wear your comfy pants and slippers. We move at a lightning pace! Only successful candidates will be contacted. If you haven\'t been invited via email to complete an interview kit within 5 business days after submitting your application, unfortunately you haven\'t been successful this time around!
"""

In [ ]:
result = model.generate_content(resume)
result.text

In [ ]:
stop

In [ ]:
result = model.generate_content('123455')
result.text

In [ ]:
result = model.generate_content('four')
result.text

In [ ]:
result = model.generate_content('quatre') # French 4
result.text                               # French 5 is "cinq"

In [ ]:
result = model.generate_content('III')    # Roman numeral 3
result.text                               # Roman numeral 4 is IV

In [ ]:
result = model.generate_content('七')  # Japanese 7
result.text                            # Japanese 8 is 八!

It really seems to have picked up the task despite the limited examples, but "next" is a simple concept, see the [tuning guide](https://ai.google.dev/gemini-api/docs/model-tuning) for more guidance on improving performance.

## Update the description

You can update the description of your tuned model any time using the `genai.update_tuned_model` method.

In [ ]:
genai.update_tuned_model(f'tunedModels/{name}', {"description":"This is my model."});

In [ ]:
model = genai.get_tuned_model(f'tunedModels/{name}')

model.description

## Delete the model

You can clean up your tuned model list by deleting models you no longer need. Use the `genai.delete_tuned_model` method to delete a model. If you canceled any tuning jobs, you may want to delete those as their performance may be unpredictable.

In [ ]:
genai.delete_tuned_model(f'tunedModels/{name}')

The model no longer exists:

In [ ]:
try:
  m = genai.get_tuned_model(f'tunedModels/{name}')
  print(m)
except Exception as e:
  print(f"{type(e)}: {e}")